# Face Detection with OpenCV

**Facial recognition** is an artificial intelligence technique, used in biometrics to identify or verify the identity of a person starting from one or more images that portray them.

Recognition usually occurs through digital image processing techniques, ignoring everything that does not represent a face, such as buildings, trees, bodies, which are usually called backgrounds. It can be said that it is a pattern recognition, where the pattern to be recognized is the human face.

To facilitate the identification of a face, the first systems took into account that a human face is composed of two eyes, a nose and a mouth. The most recent systems, however, are able to recognize a person even if their face is rotated, or in any case not in frontal vision.


https://www.kaggle.com/code/serkanpeldek/face-detection-with-opencv

In this study, face detection is performed **from the images in the LFW data set**. 

### Face Detector Class

**The ```haarcascade_frontalface_default.xml```** file is **an XML template that contains pre-trained data for face detection using OpenCV Haar Cascade classifiers**. This type of model exploits Haar features (feature-based) and a cascade classification technique, useful for real-time processing and widely used for face detection, even on devices with limited resources.

In addition to haarcascade_frontalface_default.xml, OpenCV offers additional Haar Cascade models specifically for face and face part detection:

- ```haarcascade_frontalface_alt.xml``` and ```haarcascade_frontalface_alt2.xml``` – Alternative versions to improve accuracy over various face tilt angles.
- ```haarcascade_profileface.xml``` – Detect the face in profile.
- ```haarcascade_eye.xml``` and ```haarcascade_eye_tree_eyeglasses.xml``` – For eye detection, even with glasses.
- ```haarcascade_smile.xml``` – Specifically for smile detection.

In [8]:
import cv2
import numpy as np
import os

In [4]:
cascade_dir = "/home/pierluigi/Documents/models/Face Detenction/haarcascades"

In [5]:
cascade_files = [
    "haarcascade_eye_tree_eyeglasses.xml", "haarcascade_eye.xml",
    "haarcascade_frontalface_alt2.xml", "haarcascade_frontalface_alt_tree.xml",
    "haarcascade_frontalface_alt.xml", "haarcascade_frontalface_default.xml",
    "haarcascade_fullbody.xml", "haarcascade_lefteye_2splits.xml",
    "haarcascade_lowerbody.xml", "haarcascade_mcs_eyepair_big.xml",
    "haarcascade_mcs_eyepair_small.xml", "haarcascade_mcs_leftear.xml",
    "haarcascade_mcs_lefteye.xml", "haarcascade_mcs_mouth.xml",
    "haarcascade_mcs_nose.xml", "haarcascade_mcs_rightear.xml",
    "haarcascade_mcs_righteye.xml", "haarcascade_mcs_upperbody.xml",
    "haarcascade_profileface.xml", "haarcascade_righteye_2splits.xml",
    "haarcascade_smile.xml", "haarcascade_upperbody.xml"
]

In [6]:
# Dizionario per contenere i detector
detectors = {}

# Carica ciascun file XML come CascadeClassifier e salva nel dizionario
for file in cascade_files:
    file_path = os.path.join(cascade_dir, file)
    cascade_name = file.split('.')[0]  # Usa il nome senza estensione come chiave
    detectors[cascade_name] = cv2.CascadeClassifier(file_path)
    if detectors[cascade_name].empty():
        print(f"Warning: unable to load classifier from {file_path}")

# Verifica il caricamento dei detector
for name, detector in detectors.items():
    print(f"{name} loaded: {not detector.empty()}")

haarcascade_eye_tree_eyeglasses loaded: True
haarcascade_eye loaded: True
haarcascade_frontalface_alt2 loaded: True
haarcascade_frontalface_alt_tree loaded: True
haarcascade_frontalface_alt loaded: True
haarcascade_frontalface_default loaded: True
haarcascade_fullbody loaded: True
haarcascade_lefteye_2splits loaded: True
haarcascade_lowerbody loaded: True
haarcascade_mcs_eyepair_big loaded: True
haarcascade_mcs_eyepair_small loaded: True
haarcascade_mcs_leftear loaded: True
haarcascade_mcs_lefteye loaded: True
haarcascade_mcs_mouth loaded: True
haarcascade_mcs_nose loaded: True
haarcascade_mcs_rightear loaded: True
haarcascade_mcs_righteye loaded: True
haarcascade_mcs_upperbody loaded: True
haarcascade_profileface loaded: True
haarcascade_righteye_2splits loaded: True
haarcascade_smile loaded: True
haarcascade_upperbody loaded: True


The code defines a **FetchLFW class, which is used to load and manage a random number of images from the Labeled Faces in the Wild (LFW) dataset**, a dataset containing images of faces commonly used to train and test face recognition models.

In [9]:
class FetchLFW:
    def __init__(self, path):
        self.path = path  # Percorso alla cartella contenente le immagini
    
    def _initialize(self, dim):
        self.dim_of_photo_gallery = dim
        self.number_of_images = self.dim_of_photo_gallery * self.dim_of_photo_gallery
        
        image_files = [f for f in os.listdir(self.path) if f.endswith(".jpg")]
        total_number_of_images = len(image_files)
        
        # Controllo sul numero di immagini richieste rispetto a quelle disponibili
        if self.number_of_images > total_number_of_images:
            raise ValueError(f"Richiesto {self.number_of_images} immagini, ma ne sono disponibili solo {total_number_of_images}")
        
        self.random_face_indexes = np.arange(total_number_of_images)
        np.random.shuffle(self.random_face_indexes)
        self.n_random_face_indexes = self.random_face_indexes[:self.number_of_images]
        
        self.image_files = [image_files[i] for i in self.n_random_face_indexes]
        
    def get_lfw_images(self, dim=5):
        self._initialize(dim)
        self.lfw_images = self._get_images()
        return self.lfw_images
        
    def _get_images(self):
        image_list = []
        counter = 0
        for filename in self.image_files:
            file_path = os.path.join(self.path, filename)
            image = cv2.imread(file_path, cv2.IMREAD_COLOR)
            if image is not None:
                # Ridimensiona l'immagine per ridurre l'uso della memoria
                image = cv2.resize(image, None, fx=0.4, fy=0.4, interpolation=cv2.INTER_AREA)
                image_list.append(np.array(image))
            counter += 1
        
        return np.array(image_list)

In [11]:
# Esempio di utilizzo
dataset_path = "/home/pierluigi/Documents/datasets/Face Detenction/Frontalized_LFW"
fetcher = FetchLFW(dataset_path)
lfw_images = fetcher.get_lfw_images(dim=5)

ValueError: Richiesto 25 immagini, ma ne sono disponibili solo 0